In [2]:
import pandas as pd

df = pd.read_csv("../hamiltonian_evolution_rev1.csv").set_index('name')

# Compare with tket

Tket optimization is applied on top of the greedy compiler. Note that tket factors out the SWAP gates.

In [ ]:
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm_str
from pytket.passes import CliffordSimp, PauliSimp, FullPeepholeOptimise
from pytket import OpType
from qiskit.exceptions import QiskitError
from tqdm import tqdm
tqdm.pandas()

def tket_get_cost(c):
    return c.n_gates_of_type(OpType.CX) + c.n_gates_of_type(OpType.CZ) + 3*c.n_gates_of_type(OpType.SWAP)

def tket_optimize(qasm, cost):
    """
    qasm (str): input circuit
    cost (int): pre-computed circuit cost for verification
    """
    c = circuit_from_qasm_str(qasm)
    cost_before = tket_get_cost(c)
    assert cost_before == cost
    while (1):
        cost_before = tket_get_cost(c)
        CliffordSimp().apply(c)
        cost_after = tket_get_cost(c)
        if cost_after==cost_before:
            break        
    return cost_after

df['tket_cost_keep_repeating'] = df.progress_apply(
    lambda row: tket_optimize(row['compiled'], row['compiled_cost']),
    axis=1
)

 13%|█▎        | 300/2264 [01:09<12:35,  2.60it/s]